# Splunk App for Data Science and Deep Learning - Hashing Encoder Distances: Distance For Tokenzied Strings from Reference

Hashing Encoder Distances: Distance For Tokenzied Strings from Reference

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [ ]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import re
#from hashlib import md5
import hashlib
from base64 import encode
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

### Option 2 - push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("hashing_encoder_distance")

In [ ]:
df

In [ ]:
param

## Stage 2 - create and initialize a model

In [ ]:
def init(df,param):
    model = {}
    
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

## Stage 3 - fit the model

In [ ]:
def fit(model,df,param):
    return df


In [ ]:
returns = fit(model,df,param)
print(returns)

## Stage 4 - apply the model

In [ ]:
def apply(model,df,param):

    
    field_name = param['feature_variables'][0]
    
    comparison_string = param['options']['params']['comparison_string']
    
    try:
        vector_length=int(param['options']['params']['vector_length'])
    except:
        vector_length=16
        
    try:
        return_vector=int(param['options']['params']['return_vector'])
    except:
        return_vector=0
    
    
    try:
        regex=param['options']['params']['tokenizing_regex']
    except:
        regex=r'[^\w\s]'
    
    # define the regex pattern used for tokenisation
    pattern = re.compile(regex)
    
    
    # Define a new row
    new_row = {field_name: [comparison_string]}

    # Append the row
    df = pd.concat([df,pd.DataFrame(new_row)], ignore_index=True)
    
    # split results up using the regex pattern to create a list of tokens
    df['token_list'] = df[field_name].astype(str).apply(lambda x: pattern.split(x))
    # apply strip to each list item (token) to remove leading and trailing whitespace
    df['token_list'] = df['token_list'].apply(lambda x: [item.strip() for item in x])
    df['token_list'] = df['token_list'].apply(lambda row: [item for item in row if item != ''])

    # Now that we have the tokens, we need to hash them and apply hashing trick to create an embedding vector
    # define length of enbedding vector

    # initialise a new vector of length embedding_len, defined as a new field in the query results
    df['embedding'] = df.apply(lambda row: [0] * vector_length, axis=1)

    # convert tokens to indexes into embedding array
    df['index_list'] = df['token_list'].apply(lambda token_list: [int(hashlib.md5(token.encode()).hexdigest(), 16) % (vector_length) for token in token_list])
    
    for row in range(len(df['index_list'])):
        for col in df['index_list'][row]:
            df['embedding'][row][col] += 1
            
    encoded_df = df['embedding'].apply(pd.Series)
    
    row_to_subtract = encoded_df.iloc[-1]
    encoded_df_diff = (encoded_df - row_to_subtract).drop(encoded_df.index[-1])
    
    distance = pd.DataFrame(np.sqrt((encoded_df_diff**2).sum(axis=1)))
    distance.rename(columns={0: 'distance'},inplace=True)

    if return_vector==1:
        return encoded_df_diff
    else:
        return distance
    

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
result = apply(model,df,param)
print(result)

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns